In [ ]:
import pandas as pd
from pathlib import Path
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import nevergrad as ng
import plotly.express as px

from estival.model import BayesianCompartmentalModel
from estival.optimization.nevergrad import optimize_model
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget, CustomTarget
from estival.calibration import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import *
from tbdynamics.utils import build_contact_matrix
import plotly.graph_objects as go
from general_utils.parameter_utils import load_param_info
from general_utils.doc_utils import TextElement, TableElement, FigElement, add_element_to_document, \
    save_pyplot_add_to_doc, save_plotly_add_to_doc, compile_doc, generate_doc


In [ ]:
PROJECT_PATH = Path().resolve()
DATA_PATH =  PROJECT_PATH / "data"
SUPPLEMENT_PATH = PROJECT_PATH / "supplement"

In [ ]:
pd.options.plotting.backend = "plotly"
time_start = 1800
time_end = 2020
time_step = 1

doc_sections = {}
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50]

In [ ]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 227344.75719536067,
    "cdr_adjustment": 0.6,
    "progression_multiplier": 1.1,
    "infectious_seed": 1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.08,
    'smear_positive_death_rate':0.364337776897486,
    'smear_negative_death_rate': 0.027588310343242016, 
    'smear_positive_self_recovery': 0.20344728302826143,
    'smear_negative_self_recovery': 0.22723824998716693,
    'rr_progression_diabetes': 4.5
}

In [ ]:
tb_model, build_text = model.build_base_model(compartments, infectious_compartments, time_start, time_end, time_step)
add_element_to_document("Model construction", TextElement(build_text), doc_sections)
build_text

In [ ]:
start_text = model.set_starting_conditions(tb_model)
add_element_to_document("Model construction", TextElement(start_text), doc_sections)
start_text

In [ ]:
entry_text = model.add_entry_flow(tb_model)
add_element_to_document("Model construction", TextElement(entry_text), doc_sections)
entry_text

In [ ]:
ndeath_text = model.add_natural_death_flow(tb_model)
add_element_to_document("Model construction", TextElement(ndeath_text), doc_sections)
ndeath_text

In [ ]:
infect_text = model.add_infection(tb_model)
for text in infect_text:
    add_element_to_document("Model construction", TextElement(text), doc_sections)
infect_text

In [ ]:
latency_text = model.add_latency(tb_model)
for text in latency_text:
    add_element_to_document("Model construction", TextElement(text), doc_sections)
latency_text

In [ ]:
ideath_text = model.add_infect_death(tb_model)
add_element_to_document("Model construction", TextElement(ideath_text), doc_sections)
ideath_text

In [ ]:
sr_text = model.add_self_recovery(tb_model)
add_element_to_document("Model construction", TextElement(sr_text), doc_sections)
sr_text

In [ ]:
detection_text = model.add_detection(tb_model)
add_element_to_document("Model construction", TextElement(detection_text), doc_sections)
detection_text

In [ ]:
acf_text = model.add_acf(tb_model, fixed_parameters)
add_element_to_document("Model construction", TextElement(acf_text), doc_sections)
acf_text

In [ ]:
treatment_text = model.add_treatment_related_outcomes(tb_model)
for text in treatment_text:
    add_element_to_document("Model construction", TextElement(text), doc_sections)
treatment_text

In [ ]:
mfilename = "matrix.jpg"
matrix, matrix_fig_text = build_contact_matrix(age_strata, mfilename)
add_element_to_document("Mixing", FigElement(mfilename, caption=matrix_fig_text), doc_sections)

In [ ]:
age_strat, age_strat_text = model.add_age_strat(compartments, infectious_compartments, age_strata, matrix, fixed_parameters)
add_element_to_document("Model construction", TextElement(age_strat_text), doc_sections, subsection_name="Stratifications")

In [ ]:
tb_model.stratify_with(age_strat)

In [ ]:
organ, organ_text = model.add_organ_strat(fixed_parameters,infectious_compartments)
add_element_to_document("Model construction", TextElement(organ_text), doc_sections, subsection_name="Stratifications")

In [ ]:
tb_model.stratify_with(organ)

In [ ]:
#gender = model.add_gender_strat(tb_model, age_strata, compartments, fixed_parameters)

In [ ]:
model.request_output(tb_model, compartments,  latent_compartments, infectious_compartments)

In [ ]:
priors = [
    UniformPrior("start_population_size", (150000, 300000)),
    UniformPrior("contact_rate", (0.0001, 0.02)),
    #UniformPrior("infectious_seed", [100, 2000]),
    UniformPrior("rr_infection_latent", (0.2, 0.5)),
    UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
    UniformPrior("cdr_adjustment", (0.6, 1.0)),\
    UniformPrior("rr_progression_diabetes", (2.0, 6.0))
    # UniformPrior("progression_multiplier", (0.1, 2.0)),
    # UniformPrior("cdr_adjustment", [0.6, 1.0]),
    # UniformPrior("infect_death_rate_dict.smear_positive", [0.335, 0.449]),
    # UniformPrior("infect_death_rate_dict.smear_negative", [0.017, 0.035]),
    # UniformPrior("self_recovery_rate_dict.smear_positive", [0.177, 0.288]),
    # UniformPrior("self_recovery_rate_dict.smear_negative", [0.073, 0.209]),
    # UniformPrior("rr_progression_diabetes", [1, 10]),
]
pop = pd.Series({2009: 1207100, 2019: 1194300})
notif = pd.Series({2011: 1495,2012: 1485,2013: 1369,2014:1405,2015:1642, 2016:1555, 2017:1440, 2018:1468, 2019:1417})
latent = pd.Series({2016:36})

def least_squares(modelled, obs, parameters, time_weights):
    return 0.0 - (((modelled - obs) ** 2.0)).sum()
targets = [
    CustomTarget("total_population", pop, least_squares),
   # CustomTarget("notifications", notif, least_squares),
    NegativeBinomialTarget("notifications", notif, 20.0),
    CustomTarget("percentage_latent", latent, least_squares)
]
# binom_targets = [
#     NegativeBinomialTarget("total_population", pop, 20.0),
# ]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [ ]:
optimise_model = True
if optimise_model:
    print("Optimising with nevergrad \n Progression of loss function values:")
    optim_runner = optimize_model(calibration_model)
    for i in range(10):
        rec = optim_runner.minimize(100)
        print(rec.loss)
    optim_params = rec.value[1]
    params.update(optim_params)
    tb_model.run(parameters=params)
    print("Best calibration parameters found:")
    print(optim_params)

In [ ]:
tb_model.run(parameters=params)
derived_df_0 = tb_model.get_derived_outputs_df()

In [ ]:
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [1207100, 1194300],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
     "notifications": {
      "title": "Notifications",
      "output_key": "notifications",
      "times": [2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0],
      "values": [1495, 1485, 1369, 1405, 1642, 1555, 1440, 1468, 1417],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    "percentage_latent": {
      "title": "Percentage Latent",
      "output_key": "percentage_latent",
      "times": [2016.0],
      "values": [30.8],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    
    }

In [ ]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()
total_fig_name = "total.jpg"
fig2_3.write_image(SUPPLEMENT_PATH /total_fig_name)

In [ ]:
add_element_to_document("Outputs",FigElement(total_fig_name, caption="Notifications"), doc_sections)

In [ ]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="notifications",
)
notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
notif_2.update_traces(marker=dict(color="red"))
notif_plot = go.Figure(
    data=notif_1.data + notif_2.data,
)
notif_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
)
notif_plot.show()
notif_fig_name = "notifications.jpg"
notif_plot.write_image(SUPPLEMENT_PATH / notif_fig_name)

In [ ]:
add_element_to_document("Outputs",FigElement(notif_fig_name, caption="Notifications"), doc_sections)

In [ ]:
prev_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="prevalence_infectious",
)
prev_plot.show()
prev_plot.write_image(str(SUPPLEMENT_PATH) + "/prevalance.jpg")

In [ ]:
inci_plot = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="incidence",
)
inci_plot.show()
inci_plot.write_image(str(SUPPLEMENT_PATH) + "/incidence.jpg")

In [ ]:
latent_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="percentage_latent",
)
latent_2 = px.scatter(x= plots['percentage_latent']['times'], y = plots['percentage_latent']['values'])
latent_2.update_traces(marker=dict(color="red"))
latent_plot = go.Figure(
    data=latent_1.data + latent_2.data,
)
latent_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Percentage latent"
)
latent_plot.show()
latent_plot.write_image(str(SUPPLEMENT_PATH) + "/latent.jpg")

In [ ]:
supplement = generate_doc("Supplemental Appendix", "austcovid")
compile_doc(doc_sections, supplement)

In [ ]:
import multiprocessing

In [ ]:
multiprocessing.cpu_count()